In [ ]:
import ipywidgets.widgets as widgets
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.servos import get_controls
from ballsbot.manipulator import Manipulator
from ballsbot.drawing import ManipulatorPoseDrawing
from ballsbot.utils import run_as_thread, keep_rps

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

painter = ManipulatorPoseDrawing(widgets)
display(painter.get_images())

In [ ]:
car_controls = get_controls()

In [ ]:
manipulator = Manipulator(controller)

In [ ]:
def update_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=2)
        pose = manipulator.get_manipulator_pose()
        painter.update_images(pose, override_crop_half_size=550.)

run_as_thread(update_images)